# Neo4j Generative AI Workshop Example Application
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/neo4j-product-examples/genai-workshop/blob/main/genai-example-app-only.ipynb)

__This notebook is a copy of `genai-workshop.ipynb` that contains only the final section: the example application for the LLM content generator. This notebook assumes you have already run `genai-workshop.ipynb`.__

__Please note: There is no need to run this notebook for the workshop.  It exists for demo purposes only.__

## Setup

### Some Logistics
1. Make a copy of this notebook in Colab by [clicking here](https://colab.research.google.com/github/neo4j-product-examples/genai-workshop/blob/main/genai-workshop.ipynb).
2. Run the pip install below to get the necessary dependencies.  this can take a while. Then run the following cell to import relevant libraries


In [1]:
%%capture
%pip install langchain langchain-openai langchain_community openai tiktoken python-dotenv gradio neo4j

In [8]:
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import os
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.vectorstores.neo4j_vector import Neo4jVector
from langchain.graphs import Neo4jGraph
from langchain.prompts import PromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnableLambda
import gradio as gr

pd.set_option('display.max_rows', 10)
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.width', 0)

### Setup Credentials and Environment Variables

There are two things you need here.
1. Start a blank [Neo4j Sandbox](https://sandbox.neo4j.com/). Get your URI and password and plug them in below.  Do not change the Neo4j username.
2. Get your OpenAI API key.  You can use [this one](https://docs.google.com/document/d/19Lqjd0MqRs088KUVnd23ZrVU9G0OAg-53U72VrFwwms/edit) if you do not have one already.

To make this easy, you can write the credentials and env variables directly into the below cell.

In [9]:
import os

# Neo4j
NEO4J_URI = 'bolt://34.202.229.218:7687' #change this
NEO4J_PASSWORD = 'terminologies-fire-planet' #change this
NEO4J_USERNAME = 'neo4j'
AURA_DS = False

# AI
LLM = 'gpt-4o'

# OpenAI - Required when using OpenAI models
os.environ['OPENAI_API_KEY'] = 'sk-...' #change this
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [14]:
# You can skip this cell if not using a ws.env file - alternative to above
from dotenv import load_dotenv
import os

if os.path.exists('ws.env'):
    load_dotenv('ws.env', override=True)

    # Neo4j
    NEO4J_URI = os.getenv('NEO4J_URI')
    NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
    NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
    AURA_DS = eval(os.getenv('AURA_DS').title())

    # AI
    LLM = 'gpt-4o'
    OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

## LLM For Generating Grounded Content

Let's use an LLM to automatically generate content for targeted marketing campaigns grounded with our knowledge graph using the above tools.
Here is a quick example for generating promotional messages, but you can create all sorts of content with this!

For our first message, let's consider a scenario where a user recently searched for products, but perhaps didn't commit to a purchase yet. We now want to send a message to promote relevant products.

In [15]:
from langchain_openai import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings()
embedding_dimension = 1536

In [16]:
# Import relevant libraries
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.schema import StrOutputParser

#Instantiate LLM
llm = ChatOpenAI(temperature=0, model_name=LLM, streaming=True)

### Create Knowledge Graph Stores for Retrieval

To ground our content generation, we need to define retrievers to pull information from our knowledge graph.  Let's make two stores:
1. Personalized Search Retriever (`kg_personalized_search`): Based on recent customer searches and purchase history, pull relevant products.
2. Recommendations retriever (`kg_recommendations_app`): Based on our Graph ML, what else can we recommend to them to pair with the relevant products?


In [23]:
# We will use a mock URL for our sources in the metadata
kg_personalized_search_store = Neo4jVector.from_existing_index(
    embedding=embedding_model,
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    index_name='product_text_embeddings',
    retrieval_query="""
        WITH node AS product, score AS searchScore

        OPTIONAL MATCH(product)<-[:VARIANT_OF]-(:Article)<-[:PURCHASED]-(:Customer)
        -[:PURCHASED]->(a:Article)<-[:PURCHASED]-(:Customer {customerId: $customerId})
        WITH count(a) AS purchaseScore, product, searchScore
        RETURN product.text + '\nurl: ' + product.url  AS text,
            (1.0+purchaseScore)*searchScore AS score,
            {source: product.url} AS metadata
        ORDER BY purchaseScore DESC, searchScore DESC LIMIT 10
    """
)

# This will be a function so we can change per customer id
def kg_personalized_search(search_prompt, customer_id, k=100):
    docs = kg_personalized_search_store.similarity_search(search_prompt, k, params={'customerId': customer_id})
    return "\n\n".join([d.page_content for d in docs])

# Use the same personalized recommendations as above but with a smaller limit
kg = Neo4jGraph(url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD)
def kg_recommendations_app(customer_id, k=30):
    res = kg.query("""
    MATCH(:Customer {customerId:$customerId})-[:PURCHASED]->(:Article)
    -[r:CUSTOMERS_ALSO_LIKE]->(:Article)-[:VARIANT_OF]->(product)
    RETURN product.text + '\nurl: ' + product.url   AS text,
        sum(r.score) AS recommenderScore
    ORDER BY recommenderScore DESC LIMIT $k
    """, params={'customerId': customer_id, 'k':k})

    return "\n\n".join([d['text'] for d in res])

### Prompt Engineering

Now let's define our prompt. We will accept multiple parameters and provide detailed instructions to the LLM to condition the response based of retrieved data, customer interests, and time of year.


In [24]:
prompt = PromptTemplate.from_template('You are a personal assistant named Sally '
'for a fashion, home, and beauty company called HRM.'
'write an engaging email to {customerName}, one of your customers, '
'to promote and summarize products relevant for them given: '
'- The current season / time of year: {timeOfYear}'
'- Recent searches/interests: {customerInterests}'
'Please only mention the products listed below. '
'Do not come up with or add any new products to the list.'
'Each product comes with an https `url` field. '
'Make sure to provide that https url with descriptive name text '
'in markdown for each product.'
'''

# RelevantProducts:
These are products from the HRM store the customer may be interested in based
on their recent searches/interests: {customerInterests}
{searchProds}

# Customer May Also Be Interested In the following
The below candidates are recommended based on the shared purchase patterns of
other customers in the HRM database.
Select the best 4 to 5 product subset from the context that best match the
time of year: {timeOfYear} and to pair with the RelevantProducts above.
For example, even if scarfs are listed here, they may not be appropriate for a
summer time of year so best not to include those.
{recProds}
''')

### Create a Chain

Now let's put a chain together that will leverage the retrievers, prompt, and LLM model. This is where Langchain shines, putting RAG together in a simple way.

In addition to the personalized search and recommendations context, we will allow for some other parameters.

1. `timeOfYear`: The time of year as a date, season, month, etc. so the LLM can tailor the language appropriately.
2. `customerName`: Ordinarily, this can be pulled from the DB, but it has been scrubbed to maintain anonymity, so we will provide our own name here.

You can potentially add other creative parameters here to help the LLM write relevant messages.


In [25]:
chain = ({'searchProds': (lambda x:kg_personalized_search(x['customerInterests'], x['customerId'])),
          'recProds': (lambda x:kg_recommendations_app(x['customerId'])),
          'customerName': lambda x:x['customerName'],
          'timeOfYear': lambda x:x['timeOfYear'],
          "customerInterests":  lambda x:x['customerInterests']}
         | prompt
         | llm
         | StrOutputParser())

### Example Runs

In [26]:
# example inputs
CUSTOMER_ID = "daae10780ecd14990ea190a1e9917da33fe96cd8cfa5e80b67b4600171aa77e0"
search_prompt = 'denim jeans'

In [27]:
#print(chain.invoke({'searchPrompt':search_prompt, 'customerId':CUSTOMER_ID, 'customerName':'Alex Smith', 'timeOfYear':'Feb, 2024'}))

#### Inspecting the Prompt Sent to the LLM
In the above run, the LLM should only be using results from our Neo4j database to populate recommendations. Run the below cell to see the final prompt that was sent to the LLM.

In [28]:
def format_final_prompt(x):
    return f'''=== Prompt to send to LLM ===
   {x.to_string()}
   === End Prompt ===
   '''
chain_print_prompt = ({'searchProds': (lambda x:kg_personalized_search(x['customerInterests'], x['customerId'])),
                       'recProds': (lambda x:kg_recommendations_app(x['customerId'])),
                       'customerName': lambda x:x['customerName'],
                       'timeOfYear': lambda x:x['timeOfYear'],
                       "customerInterests":  lambda x:x['customerInterests']}
                      | prompt
                      | format_final_prompt
                      | StrOutputParser())

print( chain_print_prompt.invoke({
    'customerInterests':search_prompt,
    'customerId':CUSTOMER_ID,
    'customerName':'Alex Smith',
    'timeOfYear':'Feb, 2024'}))

=== Prompt to send to LLM ===
   You are a personal assistant named Sally for a fashion, home, and beauty company called HRM.write an engaging email to Alex Smith, one of your customers, to promote and summarize products relevant for them given: - The current season / time of year: Feb, 2024- Recent searches/interests: denim jeansPlease only mention the products listed below. Do not come up with or add any new products to the list.Each product comes with an https `url` field. Make sure to provide that https url with descriptive name text in markdown for each product.

# RelevantProducts:
These are products from the HRM store the customer may be interested in based
on their recent searches/interests: denim jeans
##Product
Name: Rachel HW Denim TRS
Type: Trousers
Group: Garment Lower body
Garment Type: Trousers
Description: 5-pocket, ankle-length jeans in washed stretch denim in a relaxed fit with a high waist, zip fly and button and straight legs with cut-off, raw-edge hems.
url: https:

Feel free to experiment and try more!

In [29]:
#print(chain.invoke({'customerInterests':"western boots", 'customerId':CUSTOMER_ID, 'customerName':'Alex Smith', 'timeOfYear':'July, 2024'}))

### Demo App
Now let’s use the above tools to create a demo app with Gradio.  We will need to make a couple more functions, but otherwise easy to fire up from a Notebook!

In [30]:
# create multiple demo examples to try
examples = [
    [
        CUSTOMER_ID,
        'June, 2024',
        'Alex Smith',
        'denim jeans'
    ],
    [
        CUSTOMER_ID,
        'July, 2024',
        'Alex Smith',
        'western boots'
    ],
    [
        '819f4eab1fd76b932fd403ae9f427de8eb9c5b64411d763bb26b5c8c3c30f16f',
        'June, 2024',
        'Robin Fischer',
        'denim jeans'
    ],
    [
        '44b0898ecce6cc1268dfdb0f91e053db014b973f67e34ed8ae28211410910693',
        'Feb, 2024',
        'Chris Johnson',
        'Oversized Sweaters'
    ],
    [
        '819f4eab1fd76b932fd403ae9f427de8eb9c5b64411d763bb26b5c8c3c30f16f',
        'Feb, 2024',
        'Robin Fischer',
        'denim jeans'
    ],
    [
        CUSTOMER_ID,
        'Feb, 2024',
        'Alex Smith',
        'oversized sweaters'
    ],
]

In [31]:
import gradio as gr

def message_generator(*x):
    return chain.invoke({'customerInterests':x[3],
                         'customerId':x[0],
                         'customerName':x[2],
                         'timeOfYear': x[1]})

customer_id = gr.Textbox(value=CUSTOMER_ID, label="Customer ID")
time_of_year = gr.Textbox(value="June, 2024", label="Time Of Year")
search_prompt_txt = gr.Textbox(value='denim jeans', label="Customer Interests(s)")
customer_name = gr.Textbox(value='Alex Smith', label="Customer Name")
message_result = gr.Markdown( label="Message")

demo = gr.Interface(fn=message_generator,
                    inputs=[customer_id, time_of_year, customer_name, search_prompt_txt],
                    outputs=message_result,
                    examples=examples,
                    title="🪄 Message Generator 🥳")

if not os.getenv('AUTOMATED_RUN') == "true":
    demo.launch(share=True, debug=True)

Running on local URL:  http://127.0.0.1:7861
IMPORTANT: You are using gradio version 4.20.0, however version 4.29.0 is available, please upgrade.
--------

Could not create share link. Missing file: /Users/zachblumenfeld/opt/anaconda3/envs/genai-workshop/lib/python3.10/site-packages/gradio/frpc_darwin_arm64_v0.2. 

Please check your internet connection. This can happen if your antivirus software blocks the download of this file. You can install manually by following these steps: 

1. Download this file: https://cdn-media.huggingface.co/frpc-gradio-0.2/frpc_darwin_arm64
2. Rename the downloaded file to: frpc_darwin_arm64_v0.2
3. Move the file to this location: /Users/zachblumenfeld/opt/anaconda3/envs/genai-workshop/lib/python3.10/site-packages/gradio


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7861 <> None
